In [1]:
# (hypothetical) state populations
p = [27744, 25178, 19951, 14610, 9225, 3292]

# number of states
n = len(p)

# total country population
p_total = sum( p[i] for i in range(n) )

# total number of seats to distribute
k = 36

# state quotas
q = [ k * p[i] / p_total for i in range(n) ]

print("Populations:",p)
print("Quotas:",q)

Populations: [27744, 25178, 19951, 14610, 9225, 3292]
Quotas: [9.98784, 9.06408, 7.18236, 5.2596, 3.321, 1.18512]


In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
# create model object
m = gp.Model()

# create integer vars x, with x[i] being the number of seats for state i
x = m.addVars( n, vtype=GRB.INTEGER )

# distribute k seats
m.addConstr( gp.quicksum( x[i] for i in range(n) ) == k )

# each state gets at least one seat
for i in range(n):
    x[i].LB = 1

m.update()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-06-27
Using license file C:\Users\buchanan\gurobi.lic


In [4]:
# Variables y_i introduced to linearize the terms | x_i - q_i | 
y = m.addVars( n )
m.addConstrs( x[i] - q[i] <= y[i] for i in range(n) )
m.addConstrs( q[i] - x[i] <= y[i] for i in range(n) )

# Add objective function for Hamilton's method: min sum_i |x_i-q_i|
m.setObjective( gp.quicksum( y[i] for i in range(n) ), GRB.MINIMIZE )

m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13 rows, 12 columns and 30 nonzeros
Model fingerprint: 0x0ffe6cee
Variable types: 6 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve time: 0.00s
Presolved: 13 rows, 12 columns, 30 nonzeros
Variable types: 6 continuous, 6 integer (0 binary)

Root relaxation: objective 1.508506e-01, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.15085    0    6          -    0.15085      -     -    0s
H    0     0                      59.6297600    0.15085   100%     -    0s
H    0     0                       1.3823200    0.15085  89.1%     -    0s
     0  

In [5]:
sol = [ x[i].x for i in range(n) ]
print("Optimal solution :", sol)
print("Compare to quotas:", q)

Optimal solution : [10.0, 9.0, 7.0, 5.0, 4.0, 1.0]
Compare to quotas: [9.98784, 9.06408, 7.18236, 5.2596, 3.321, 1.18512]
